In [1]:
%cd ..
%load_ext autoreload
%autoreload 2


/home/samuel.assis/MatchImm/MatchImmNet


In [5]:
from pyGeoMatchImm.embeddings.positional import PositionalCDREmbedder
import torch



In [17]:
chains = ['E', 'E', 'D', 'D', 'E', 'D']
residues = torch.tensor([30, 31, 110, 85, 60, 115])

embedder = PositionalCDREmbedder()

embedder(residues, chains)


tensor([4, 4, 0, 0, 4, 0])
resids_indices: tensor([0, 0, 2, 3, 1, 2])
resids_indices after adding chains: tensor([4, 4, 2, 3, 5, 2])


tensor([[[ 1.1246, -0.7733, -0.1837, -0.6619, -1.3976, -0.6462,  0.1789,
          -0.6390,  0.0596]],

        [[ 1.1246, -0.7733, -0.1837, -0.6619, -1.3976, -0.6462,  0.1789,
          -0.6390,  0.0596]],

        [[-1.3016, -1.5554, -0.4954, -0.3434, -0.8668,  0.4128,  0.8613,
           0.1827,  0.8377]],

        [[ 0.1985, -1.5439, -1.6705,  0.5953,  0.5069,  1.6847,  1.0989,
          -1.5842,  0.6124]],

        [[-0.5731,  1.0145,  1.8119,  1.7433, -1.4543, -0.6654, -2.6007,
           0.5845, -1.3752]],

        [[-1.3016, -1.5554, -0.4954, -0.3434, -0.8668,  0.4128,  0.8613,
           0.1827,  0.8377]]], grad_fn=<EmbeddingBackward0>)